In [17]:
import requests
import json
import re
from bs4 import BeautifulSoup

要得到：
- 专辑/歌单名，tag，订阅数
- results:(歌曲id，歌曲名称，歌手名字，song['popularity']): song_info

# 获取所有歌单ID

In [18]:
def get_page(pages):
    offset_list = []
    for i in range(pages):
        offset_list.append(35*i)
        return offset_list

In [19]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
            'Referer': 'http://music.163.com/'}

album_ids = []
#extract album listnames
def extract_albumlist(offset_list):
    for i in offset_list:
        url = 'https://music.163.com/discover/playlist/?order=hot&cat=%E5%85%A8%E9%83%A8&limit=35&offset='+str(i)
        s = requests.session()
        s = BeautifulSoup(s.get(url,headers = headers).content)
        # url中要去掉#，#为锚点，代表页内跳转。
        album_list = []
        for i in s.find_all('a' , href = re.compile("^/playlist\?id=[0-9]+")):
            album_list.append(i['href'].split('=')[-1])
    return set(album_list)

In [20]:
# pages = 10
# album_lists = extract_albumlist(get_page(pages))
# return album ids of many pages

# 获取歌单内信息

In [21]:
# 网易云用了框架frame，所以你右键查看源代码的时候，有两个选择，选择框架源代码就可以看到你想要的内容了。
# play_url = 'http://music.163.com/playlist?id=317113395'
# id is album-id
# play_url = 'http://music.163.com/playlist?id='+str(id)

In [22]:
album_id = []
album_name = []
title = []
tags = []
songlist = []
def extract_maininfo(albumid):
    play_url = 'http://music.163.com/playlist?id='+str(albumid)
    s = requests.session()
    s = BeautifulSoup(s.get(play_url,headers = headers).content)
    # album-id
    album_id.append(play_url.split('=')[-1])
    # album_name !!!
    try:
        album_name.append(s.title.split(' - ')[-1])
    except Exception:
        album_name.append('unkonwn')
    # song-name
    title.append(s.title.text.split('-')[0])
    # tags
    tag_all = s.find_all('a',{'class':'u-tag'})
    for i in tag_all:
        tags.append(i.text)
#     songlist.append(extract_detailinfo(play_url))
    return album_id,album_name,title,tags

In [23]:
# get "song_ids,song_name,singer,popularity" info from each album
albumids = []
song_ids = []
song_name = []
singer = []
popularity = []
timestamp = []
def extract_detailinfo(play_url,albumid): 
    play_url = play_url + str(albumid)
    s = requests.session()
    s = BeautifulSoup(s.get(play_url,headers = headers).content)
    # <ul class="f-hide">，内含很多<a href>,所以用循环
    main = s.find('ul',{'class':'f-hide'})
    for music in main.find_all('a'):
        album_id,album_name,title,tags = extract_maininfo(albumid)
        albumids.append(album_id)
        # timestamp
        time = s.find('span',{'class':'time s-fc4'})
        time = time.text.split("\xa0")[0]
        # !!! extend字符串时，会被拆成一个个的字符
        timestamp.append(str(time))
        song_id = music['href'].split('=')[-1]
        song_ids.append(song_id)
        song_name.append(music.text) 
        # 歌手无法在source-page中得到,没有显示歌手的名字,因此要点开song_id获取歌手的名字
        ssinger = get_singer(song_id)
        # if ssinger 存在
        if ssinger:
            singer.append(ssinger)
        else:
            singer.append('error')
        # 每一个album中popularity都一样，都是播放数
        pop = s.find(id='play-count').text
        popularity.append(pop)
    return albumids,timestamp,song_ids,song_name,singer,popularity

In [24]:
# https://github.com/airbnb/enzyme/issues/1349
# that `&nbsp;` was translated to `\xa0`
play_url = 'https://music.163.com/playlist?id=2618676816'
s = requests.session()
s = BeautifulSoup(s.get(play_url,headers = headers).content)
time = s.find('span',{'class':'time s-fc4'})
# !!! .text
# time = time.text.split("&nbsp;")[0]
time = time.text.split("\xa0")[0]
time

'2019-01-10'

In [25]:
# get singer's name
def get_singer(song_id):
    url = 'https://music.163.com/song?id='+str(song_id)
    s = requests.session()
    s = BeautifulSoup(s.get(url,headers = headers).content)
    # 但是换了其它的song_id就什么也得不到了,但是其页面是可以得到的，原因可能是因为有重复的des s-fc4 &s-fc7 class，因此程序无从选择，就返回了None
    # singer = s.find('p',{'class':'des s-fc4'}).find('span',{'class':'s-fc7'})
    singer = s.title.text.split(' - ')[1]
    return singer

In [26]:
# get_singer(41500032) int型也对
# 但是换了其它的song_id就什么也得不到了？其页面是可以得到的啊。。。
get_singer('29816902')

'荷尔蒙小姐'

In [27]:
# extract_detailinfo('http://music.163.com/playlist?id=317113395')

In [28]:
# 一直报错'NoneType' object has no attribute 'text'：singer = singer.find('span',{'class':'s-fc7'}).text
# print表示问题是由song_id组成的URL有问题，造成了None值而导致的
# 直接用.tag就解决了这个问题

# 整理成符合movielens的格式，存为json

In [29]:
pages = 2
album_lists = extract_albumlist(get_page(pages))

In [30]:
album_lists

{'2127607940',
 '2283573424',
 '2353275887',
 '2369772502',
 '2376558926',
 '2395304063',
 '2399099637',
 '2407258591',
 '2428607226',
 '2429537442',
 '2431814627',
 '2434987000',
 '2444897054',
 '2445945181',
 '2451877636',
 '2452313143',
 '2468971321',
 '2489592180',
 '2498530665',
 '2512092824',
 '2514836260',
 '2520736219',
 '2537027876',
 '2537791164',
 '2538757921',
 '2539003950',
 '2570049352',
 '2571114602',
 '2578068117',
 '2578520695',
 '2582768742',
 '2600419026',
 '2639030486',
 '2642577208',
 '2658697247'}

In [31]:
# work
albumids,time,song_ids,song_name,singer,popularity = extract_detailinfo('https://music.163.com/playlist?id=','2127607940')

In [34]:
# return movie lens: user, item, rating, timestamp
# albumid, songid, popularity, timestamp
album = []
songid = []
score = []
timestamp = []
for albumid in album_lists:
    albumids,time,song_ids,song_name,singer,popularity = extract_detailinfo('https://music.163.com/playlist?id=',albumid)
    # 不要重名哟，之前起了album_id就无法run，总被当作之前的str而不是新赋的list
    album.extend(albumids)
    songid.extend(song_ids)
    score.extend(popularity)
    timestamp.extend(time)

ConnectionError: HTTPSConnectionPool(host='music.163.com', port=443): Max retries exceeded with url: /song?id=546279760 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x111637240>: Failed to establish a new connection: [Errno 50] Network is down',))